## The Bot

In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge,EdgeOptions

def get_tweet_data(card):
    """Extract data from tweets"""
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    """Sponsored content has no date"""
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment+responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    """Consolidate into a tuple"""
    tweet = (username,handle,postdate,text,reply_cnt,retweet_cnt,like_cnt)
    return tweet

#create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

#twitter login
driver.get('http://www.twitter.com/login')
#in case the page opens too slow
sleep(5)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
my_username = ''
username.send_keys(my_username)
my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

sleep(5)
#find searchbox and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('#Dogecoin')
search_input.send_keys(Keys.RETURN)

#navigate to 'Latest' tab for historical data
driver.find_element_by_link_text('Latest').click()

#get all tweets on the page 
data = []
tweet_ids = set()

#check if the end is reached
last_position = driver.execute_script("return window.pageYOffset;")

#double check before ending the loop
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    """Auto scroll"""
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        sleep(1)
        current_position = driver.execute_script("return window.pageYOffset;")
        if last_position == current_position:
            scroll_attempt += 1
            
            #end of scroll condition
            if scroll_attempt >=3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = current_position
            break

## Store the Data

In [ ]:
with open('dogecoin_tweets.csv','w',newline='',encoding='utf-8') as f:
    header = ['Username','Handle','Timestamp','Text','Replies','Retweets','Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

## Read the Data

In [2]:
import pandas as pd

dir = "C:/Users/Yang Yue/Documents/GitHub/Webscraping Practice/"
tweets_df = pd.read_csv(dir+"dogecoin_tweets.csv")
tweets_df.head(10)

,Username,Handle,Timestamp,Text,Replies,Retweets,Likes
0,Brett Murphy,@bmurphypointman,2021-04-22T01:48:27.000Z,#investor #DigitalMarketing #SEO #Bitcoin\n #n...,NaN,NaN,NaN
1,C LaPenta,@Chris_LaPenta,2021-04-22T01:48:26.000Z,Replying to \n@thekingdom#dogecoin #DogecoinRi...,NaN,NaN,NaN
2,Ego,@StevenMauro14,2021-04-22T01:48:24.000Z,Replying to \n@davidgokhshteinWait till June. ...,NaN,NaN,NaN
3,Brett Murphy,@bmurphypointman,2021-04-22T01:48:22.000Z,#affiliatemarketing #earnmoney #affiliateprogr...,NaN,NaN,NaN
4,Darkness82,@Justiz1282,2021-04-22T01:48:21.000Z,Replying to \n@DiGiornoHeard you guys are next...,NaN,NaN,NaN
5,Brett Murphy,@bmurphypointman,2021-04-22T01:48:20.000Z,#shoppingstar #linkedin #twitter #facebook #in...,NaN,NaN,NaN
6,Issac Johnson,@Izzak_83,2021-04-22T01:48:19.000Z,*googles how much I need to save for \n@Tesla\...,NaN,NaN,1
7,fallinsky15,@fallinsky15,2021-04-22T01:48:13.000Z,Replying to \n@dogecoin_riseYou know it’s gett...,NaN,NaN,NaN
8,Brett Murphy,@bmurphypointman,2021-04-22T01:48:12.000Z,#freelance #freelancer #freelancing #affiliate...,NaN,NaN,NaN
9,Dori Chan,@tutiion,2021-04-22T01:48:12.000Z,Look at #dogecoin! What a cutieQuote Tweet\nBl...,NaN,NaN,NaN
